In [140]:
import pandas as pd
import numpy as np

Importo la base de datos de los pacientes, recogida en SPSS y pasada posteriormente a documento csv. Para su analisis

In [141]:
datos=pd.read_csv("Analisis_ano.csv",sep=";")

In [142]:
datos.head()

,Médico,Tipo_cápsula,Fecha_procedimiento,ID,Sexo,Nacimiento,Edad,Cintura,IMC,Tiempo_estudio,...,Patency,No_aporta_previo,Eco_abdominal,Rx_abdomen,Entero_TAC,Entero_RM,Tránsito_intes,Endoscopia_previa,Intro_cápsula,Complicaciones
0,1,1,5/14/2012,1,1,6/24/1925,86,,,,...,0,1,0,0,0,0,0,0,0,0
1,1,1,9/5/2012,2,1,1/12/1953,59,,,,...,0,1,0,0,0,0,0,0,0,0
2,1,1,10/17/2012,3,1,2/21/1951,61,,,,...,0,1,0,0,0,0,0,0,0,0
3,1,1,12/14/2013,4,1,10/1/1950,63,,,,...,0,1,0,0,0,0,0,0,0,0
4,1,1,10/25/2013,5,0,7/4/1935,78,,,,...,0,1,0,0,0,0,0,2,0,0


In [143]:
datos.shape

(251, 150)

In [144]:
datos.dtypes[:]

Médico                  int64
Tipo_cápsula            int64
Fecha_procedimiento    object
ID                      int64
Sexo                    int64
                        ...  
Entero_RM               int64
Tránsito_intes          int64
Endoscopia_previa      object
Intro_cápsula           int64
Complicaciones         object
Length: 150, dtype: object

In [145]:
columnas=datos.columns

In [146]:
incompletas=[]
for e in columnas:
    if datos[e].isnull().sum()>=1:
        incompletas.append(e)
print(incompletas)

[]


Como se ve no hay ninguna variable, con valores nulos en toda la base de datos

In [152]:
columnas=datos.columns
columnas.tolist()

['Médico',
 'Tipo_cápsula',
 'Fecha_procedimiento',
 'ID',
 'Sexo',
 'Nacimiento',
 'Edad',
 'Cintura',
 'IMC',
 'Tiempo_estudio',
 'T_estómago',
 'T_ID_total',
 'T_colon_total',
 'T_expulsión',
 'Expulsa_cápsula',
 'HDOO_oculta',
 'HDOO_mani',
 'Crohn',
 'Alt_tránsito',
 'Anemia',
 'Dolor_abd',
 'Estreñimiento',
 'Diarrea',
 'Vómitos',
 'Pérdida_peso',
 'EII',
 'Colono_imcp',
 'Cribado',
 'Revisión',
 'Otros_indica',
 'Hernia_hiato_esof',
 'RGE_esof',
 'Trast_tránsito_esof',
 'Erosiones_esof',
 'Barret_esof',
 'Esof_eosinofilica_esof',
 'Estenosis_extr_esof',
 'No_evaluable_esof',
 'No_informado_esof',
 'Normal_esof',
 'Normal_estom',
 'No_informado_estom',
 'No_evaluable_estom',
 'Hernia_hiato_estom',
 'Trast_motilidad_estom',
 'Erosiones_estom',
 'Gastritis_estom',
 'Pólipo_estom',
 'Otros_estom',
 'No_informado_del',
 'No_evaluable_del',
 'Normal_del',
 'Hz_yeyuno',
 'Yeyunitis',
 'Erosiones_yey',
 'Telangiectasias_yey',
 'Linfagiectasias_yey',
 'Pólipo_yey',
 'Hz_íleon',
 'Ileitis

In [ ]:
#Borro la variable Dianoscitos porque sería un hallazgo compatible con la normalidad.
datos.drop(columns=['Dianocitos'],axis=0, inplace=True)

Uno en una única variable las hernias de hiato de esófago y estomago. Al ser el mismo diagnostico, simplemente puesto
en esófago o estomago según el momento de la cápsula en el que se diagnosticaba. Quedando unificada en la variable "Hernia_hiato_total"

In [177]:
def hernia(fila):
    a='Hernia_hiato_esof'
    b='Hernia_hiato_estom'
    if fila[a]==fila[b]:
        return fila[a]
    elif fila[a]<fila[b]:
        return fila[b]
    else:
        return fila[a]

In [178]:
datos["Hernia_hiato_total"]=datos.apply(lambda fila: hernia(fila),axis=1)

Compruebo que la reoorganización se ha realizado correctamente

In [208]:
comp=datos[['Hernia_hiato_esof','Hernia_hiato_estom',"Hernia_hiato_total"]].sort_values(by="Hernia_hiato_total"),
ascending=True
comp

(     Hernia_hiato_esof  Hernia_hiato_estom  Hernia_hiato_total
 0                    0                   0                   0
 150                  0                   0                   0
 151                  0                   0                   0
 152                  0                   0                   0
 153                  0                   0                   0
 ..                 ...                 ...                 ...
 84                   0                   1                   1
 212                  0                   1                   1
 210                  1                   0                   1
 188                  0                   1                   1
 125                  1                   0                   1
 
 [251 rows x 3 columns],)

# 1) Porcentaje de pacientes que vinieron con síntomas de dolor abdominal y tuvieron hallazgos en la cápsula. Diferenciando entre el centro en el que fue realizada la prueba.

Para ello, agrupo primero todas las varibles que representan hallazgos en la cápsula endoscopica

In [33]:
hallazgos=['Hernia_hiato_esof',
 'RGE_esof',
 'Trast_tránsito_esof',
 'Erosiones_esof',
 'Barret_esof',
 'Esof_eosinofilica_esof',
 'Estenosis_extr_esof',
 'Hernia_hiato_estom',
 'Trast_motilidad_estom',
 'Erosiones_estom',
 'Gastritis_estom',
 'Pólipo_estom',
 'Otros_estom',
 'Hz_yeyuno',
 'Yeyunitis',
 'Erosiones_yey',
 'Telangiectasias_yey',
 'Linfagiectasias_yey',
 'Pólipo_yey',
 'Hz_íleon',
 'Ileitis',
 'Diverticulo_íleon',
 'Erosiones_íleon',
 'Pólipo_íleon',
 'Erosiones_del',
 'Diverticulo_del',
 'Enteritis_del',
 'Pólipo_del',
 'Otros_del',
 'Hemorroides',
 'Diverticulo_col',
 'Diverticulo_C_drch',
 'Diverticulo_sig',
 'Diverticulo_C_izq',
 'Sigmoiditis',
 'Proctitis',
 'Erosiones_col',
 'Angiodisplasia_col',
 'Pólipo_col',
 'Pólipo_C_drch',
 'Pólipo_C_asce',
 'Pólipo_C_trans',
 'Pólipo_C_izq',
 'Pólipo_C_desc',
 'Pólipo_ciego',
 'Pólipo_sig',
 'Pólipo_recto',
 'Melanosis',
 'Otras_col']

In [73]:
ver=datos[datos[hallazgos]==1].index
datos.loc[ver]

,Médico,Tipo_cápsula,Fecha_procedimiento,ID,Sexo,Nacimiento,Edad,Cintura,IMC,Tiempo_estudio,...,Patency,No_aporta_previo,Eco_abdominal,Rx_abdomen,Entero_TAC,Entero_RM,Tránsito_intes,Endoscopia_previa,Intro_cápsula,Complicaciones
0,1,1,5/14/2012,1,1,6/24/1925,86,,,,...,0,1,0,0,0,0,0,0,0,0
1,1,1,9/5/2012,2,1,1/12/1953,59,,,,...,0,1,0,0,0,0,0,0,0,0
2,1,1,10/17/2012,3,1,2/21/1951,61,,,,...,0,1,0,0,0,0,0,0,0,0
3,1,1,12/14/2013,4,1,10/1/1950,63,,,,...,0,1,0,0,0,0,0,0,0,0
4,1,1,10/25/2013,5,0,7/4/1935,78,,,,...,0,1,0,0,0,0,0,2,0,0
5,1,1,10/21/2014,6,0,12/16/1943,70,,,,...,0,1,0,0,0,0,0,2,0,0
6,1,1,6/24/2012,7,1,6/17/1946,66,,,,...,0,1,0,0,0,0,0,0,0,0
7,1,1,8/8/2013,8,0,7/12/1946,67,,,,...,0,1,0,0,0,0,0,0,0,0
8,1,1,,9,0,4/4/1953,,,,,...,0,1,0,0,0,0,0,0,0,0
9,1,1,10/16/2012,10,0,7/7/1939,73,,,,...,0,1,0,0,0,0,0,0,0,0


In [112]:
cap_dol = datos[(datos['Médico']==0) & 
                (datos['Dolor_abd']==1)]

In [117]:
cap_dol["Médico"]

63     0
75     0
90     0
93     0
94     0
102    0
105    0
106    0
108    0
113    0
114    0
118    0
123    0
125    0
130    0
135    0
136    0
141    0
145    0
153    0
154    0
155    0
156    0
157    0
160    0
161    0
162    0
163    0
165    0
166    0
168    0
169    0
170    0
174    0
176    0
179    0
180    0
183    0
188    0
189    0
199    0
201    0
202    0
203    0
205    0
208    0
209    0
211    0
212    0
215    0
216    0
218    0
219    0
Name: Médico, dtype: int64

In [114]:
a=cap_dol[cap_dol[hallazgos]==1].index

In [115]:
len(a)

53

In [116]:
cap_dol.loc[a]

,Médico,Tipo_cápsula,Fecha_procedimiento,ID,Sexo,Nacimiento,Edad,Cintura,IMC,Tiempo_estudio,...,Patency,No_aporta_previo,Eco_abdominal,Rx_abdomen,Entero_TAC,Entero_RM,Tránsito_intes,Endoscopia_previa,Intro_cápsula,Complicaciones
63,0,0,10/1/2010,64,0,1/1/1982,28,,,,...,0,0,1,1,0,0,0,0,0,0
75,0,1,3/28/2011,76,1,1/1/1985,26,,,,...,0,0,1,0,0,1,0,3,0,0
90,0,1,1/18/2012,91,1,3/3/1981,30,99,"23,4075091289286",,...,0,0,1,1,0,0,0,2,0,0
93,0,1,2/7/2012,94,1,2/28/1981,30,105,"24,8583651289166",,...,0,1,0,0,0,0,0,1,0,0
94,0,1,2/23/2012,95,0,1/17/1943,69,100,"23,7118446309609",,...,0,0,1,1,0,0,0,0,0,0
102,0,1,4/3/2012,103,0,5/31/1973,38,87,"21,09375",,...,0,0,1,1,1,0,0,0,0,0
105,0,1,5/4/2012,106,0,10/28/1918,93,109,"24,65483234714",,...,1,0,1,1,0,0,0,1,0,0
106,0,1,5/23/2012,107,0,2/19/1985,27,75,"16,1058786457057",,...,0,0,1,1,0,0,0,1,0,0
108,0,1,6/25/2012,109,0,11/14/1974,37,69,"17,0392308004382",,...,0,1,0,0,0,0,0,3,0,0
113,0,1,7/11/2012,114,1,10/27/1947,64,102,"26,5337588296461",,...,0,0,0,0,1,0,0,2,0,0


In [111]:
cap_dol["Dolor_abd"]==1

63     True
75     True
90     True
93     True
94     True
102    True
105    True
106    True
108    True
113    True
114    True
118    True
123    True
125    True
130    True
135    True
136    True
141    True
145    True
153    True
154    True
155    True
156    True
157    True
160    True
161    True
162    True
163    True
165    True
166    True
168    True
169    True
170    True
174    True
176    True
179    True
180    True
183    True
188    True
189    True
199    True
201    True
202    True
203    True
205    True
208    True
209    True
211    True
212    True
215    True
216    True
218    True
219    True
Name: Dolor_abd, dtype: bool

In [82]:
datos[hallazgos]==

,Hernia_hiato_esof,RGE_esof,Trast_tránsito_esof,Erosiones_esof,Barret_esof,Esof_eosinofilica_esof,Estenosis_extr_esof,Hernia_hiato_estom,Trast_motilidad_estom,Erosiones_estom,...,Pólipo_C_asce,Pólipo_C_trans,Pólipo_C_izq,Pólipo_C_desc,Pólipo_ciego,Pólipo_sig,Pólipo_recto,Dianocitos,Melanosis,Otras_col
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,True,True,True


In [ ]:
hz_Tta=['Hernia_hiato_esof',
 'RGE_esof',
 'Erosiones_esof',
 'Hernia_hiato_estom',
 'Erosiones_estom',
 'Pólipo_estom',
 'Otros_estom',
 'Telangiectasias_yey',
 'Linfagiectasias_yey',
 'Pólipo_yey',
 'Diverticulo_íleon',
'Erosiones_del',
 'Diverticulo_del',
 'Enteritis_del',
 'Pólipo_del',
 'Otros_del',
 'Hemorroides',
 'Diverticulo_col',
 'Diverticulo_C_drch',
 'Diverticulo_sig',
 'Diverticulo_C_izq',
 'Sigmoiditis',
 'Proctitis',
 'Angiodisplasia_col',
 'Pólipo_col',
 'Pólipo_C_drch',
 'Pólipo_C_asce',
 'Pólipo_C_trans',
 'Pólipo_C_izq',
 'Pólipo_C_desc',
 'Pólipo_ciego',
 'Pólipo_sig',
 'Pólipo_recto',
 'Melanosis',
 'Otras_col']

In [ ]:
Hz_prueba=['Trast_tránsito_esof',  'Barret_esof',  'Esof_eosinofilica_esof',  'Estenosis_extr_esof',  
           'Trast_motilidad_estom','Gastritis_estom',  'Yeyunitis',  'Erosiones_yey',  'Ileitis', 
           'Erosiones_íleon',  'Pólipo_íleon',  'Erosiones_col']




